# 長期記憶（Long-term Memory）
在前面的章節中，我們使用 MemorySaver 來保存單一對話的狀態。但實際應用中，我們常需要：
- 記住使用者的偏好（跨對話）
- 儲存重要的事實和知識
- 在不同對話間共享資訊

LangGraph 的 Store 介面提供了長期記憶功能，讓 AI 可以跨 thread 記住資訊。
## Store 介面概述
Store 是一個鍵值儲存系統，與 Checkpointer 的差異：

| 特性 | Checkpointer | Store |
|------|--------------|-------|
| 用途 | 保存對話狀態 | 儲存長期記憶 |
| 範圍 | 單一 thread | 跨所有 thread |
| 生命週期 | 隨對話結束 | 永久保存 |
| 典型用途 | 對話歷史 | 使用者偏好、知識庫 |


## InMemoryStore（記憶體儲存）
LangGraph 提供 InMemoryStore 作為最簡單的 Store 實作。

In [ ]:
from langgraph.store.memory import InMemoryStore
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict

# 建立 Store
store = InMemoryStore()

class ChatState(TypedDict):
    messages: list[dict]
    user_id: str

def chat_node(state: ChatState, store):
    """聊天節點，可以存取 store"""
    user_id = state["user_id"]
    
    # 從 store 讀取使用者名稱
    user_data = store.get(("users", user_id), "profile")
    user_name = user_data.value.get("name", "訪客") if user_data else "訪客"
    
    user_msg = state["messages"][-1]["content"]
    response = f"你好 {user_name}！你說：{user_msg}"
    
    return {
        "messages": state["messages"] + [
            {"role": "assistant", "content": response}
        ]
    }

# 建立 graph
workflow = StateGraph(ChatState)
workflow.add_node("chat", chat_node)
workflow.set_entry_point("chat")
workflow.add_edge("chat", END)

checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer, store=store)

# 先儲存使用者資料到 store
store.put(("users", "user_123"), "profile", {"name": "小明", "age": 25})

# 執行對話
config = {"configurable": {"thread_id": "chat_001"}}
result = graph.invoke({
    "messages": [{"role": "user", "content": "你好"}],
    "user_id": "user_123"
}, config)

print(result["messages"][-1]["content"])

你好 小明！你說：你好


## Store 的基本操作：


In [ ]:
# 儲存資料
store.put(
    namespace=("users", "user_123"),  # 命名空間
    key="profile",                     # 鍵
    value={"name": "小明", "age": 25} # 值
)

# 讀取資料
item = store.get(("users", "user_123"), "profile")
if item:
    data = item.value # 透過 .value 取得實際資料
    print(data)  # {'name': '小明', 'age': 25}

# 搜尋資料
results = store.search(("users",))  # 回傳 Item 物件的列表
for item in results:
    print(item.value)  # 存取每個 Item 的值

# 刪除資料
store.delete(("users", "user_123"), "profile")

## Namespace 管理

Namespace 用來組織和隔離不同類型的記憶，類似資料夾結構。

In [7]:
from langgraph.store.memory import InMemoryStore

store = InMemoryStore()

# 使用 tuple 定義階層式 namespace
# 格式：(類型, 子類型, ID)

# 1. 使用者相關資料
store.put(("users", "user_123"), "profile", {
    "name": "小明",
    "email": "ming@example.com"
})

store.put(("users", "user_123"), "settings", {
    "theme": "dark",
    "language": "zh-TW"
})

# 2. 偏好設定
store.put(("preferences", "user_123"), "food", {
    "likes": ["咖啡", "壽司"],
    "dislikes": ["香菜"]
})

store.put(("preferences", "user_123"), "hobbies", {
    "items": ["閱讀", "旅遊", "攝影"]
})

# 3. 對話歷史摘要
store.put(("summaries", "user_123"), "2025-12", {
    "topics": ["AI", "旅遊規劃"],
    "interactions": 15
})

# 4. 知識庫
store.put(("knowledge", "user_123"), "fact_001", {
    "content": "喜歡喝黑咖啡",
    "category": "preference"
})

# 搜尋特定 namespace（回傳 Item 列表）
user_data = store.search(("users", "user_123"))
print("使用者資料:")
for item in user_data:
    print(f"  {item.key}: {item.value}")

preferences = store.search(("preferences", "user_123"))
print("\n偏好設定:")
for item in preferences:
    print(f"  {item.key}: {item.value}")

# 搜尋所有使用者的偏好
all_preferences = store.search(("preferences",))
print("\n所有偏好:")
for item in all_preferences:
    print(f"  {item.namespace}: {item.value}")

使用者資料:
  profile: {'name': '小明', 'email': 'ming@example.com'}
  settings: {'theme': 'dark', 'language': 'zh-TW'}

偏好設定:
  food: {'likes': ['咖啡', '壽司'], 'dislikes': ['香菜']}
  hobbies: {'items': ['閱讀', '旅遊', '攝影']}

所有偏好:
  ('preferences', 'user_123'): {'likes': ['咖啡', '壽司'], 'dislikes': ['香菜']}
  ('preferences', 'user_123'): {'items': ['閱讀', '旅遊', '攝影']}


## Namespace 設計：

In [ ]:
# 好的設計：清晰的階層
("users", user_id)                      # 使用者資料
("preferences", user_id)                # 使用者偏好
("knowledge", user_id)                  # 使用者知識
("sessions", user_id)                   # 對話會話
("analytics", user_id)                  # 分析資料

# 不好的設計：扁平化
("user_profile",)                       # 無法區分使用者
("user_123_food_preference",)           # 難以搜尋和管理

# 語意檢索

在前面的章節中，我們使用 Store 來儲存和檢索結構化資料。但當資料量增大時，我們需要更智慧的搜尋方式——語意搜尋。它能理解查詢的「意義」，而不只是關鍵字配對。

## 整合向量資料庫

### Chroma 基礎用法教學
Chroma 是一個向量資料庫，用來儲存文字的向量（embeddings），方便做相似度搜尋。基本概念：

1. 建立向量資料庫

In [ ]:
#pip install -U langchain-chroma chromadb

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
import os

os.environ["OPENAI_API_KEY"] = "xxx"

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma(
    collection_name="my_collection",
    embedding_function=embeddings
)

- collection_name：向量庫名稱
- embedding_function：用來將文字轉向量的函數

2. 新增對話

In [3]:
texts = ["今天天氣很好", "我想去旅遊"]
metadatas = [{"source": "note1"}, {"source": "note2"}]
vectorstore.add_texts(texts=texts, metadatas=metadatas)

['3a0cc831-102f-437b-a624-c2cdad872448',
 'ca46a3aa-0f90-404b-b20c-6442d9e5ed49']

3. 搜尋相似對話

In [5]:
query = "旅遊計畫"
results = vectorstore.similarity_search(query, k=1)
for r in results:
    print(r.page_content, r.metadata)


我想去旅遊 {'source': 'note2'}


- similarity_search(query, k=3, filter=...)：查詢最相似的 k 筆資料，可加條件篩選

- add_texts(texts, metadatas)：新增文本及對應 metadata

## 對話歷史語意搜尋(結合短期+長期記憶)

建立一個具備長期記憶能力的對話系統，讓 AI 助理能記住過往對話，並在未來的對話中智慧地召回相關記憶。

In [ ]:
"""
使用者輸入
   ↓
search_history_node -> 從 Chroma 找「摘要記憶」
   ↓
chat_with_history_node -> LLM 結合長期記憶與近期對話
   ↓
archive_node -> 產生 summary 並存入 Chroma / InMemoryStore
   ↓
END
"""

### 步驟 1: 建立對話歸檔類別

首先建立 ConversationArchive 類別來管理所有記憶相關的操作:

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langgraph.store.memory import InMemoryStore
from datetime import datetime

import os

os.environ["OPENAI_API_KEY"] = "xxx"

class ConversationArchive:
    def __init__(self):
        # 初始化 OpenAI 嵌入模型
        self.embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        
        # 建立 Chroma 向量資料庫
        self.vectorstore = Chroma(
            collection_name="conversation_archive",
            embedding_function=self.embeddings
        )
        
        # 建立記憶體儲存
        self.store = InMemoryStore()
        
        # 初始化 LLM 用於生成摘要
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 步驟 2: 對話摘要生成

將多輪對話濃縮成一句有意義的摘要:

In [ ]:
def generate_summary(self, messages: list[dict]) -> str:
    """
    將對話濃縮成摘要,重點放在使用者意圖和偏好
    """
    # 將訊息格式化為對話文本
    conversation = "\n".join(f"{m['role']}: {m['content']}" for m in messages)
    
    # 設計提示詞,引導 LLM 生成高品質摘要
    prompt = f"""
    請將以下對話濃縮成一句「可作為長期記憶的摘要」,
    重點放在使用者的意圖、偏好或討論主題:
    
    {conversation}
    
    摘要:
    """
    
    return self.llm.invoke(prompt).content.strip()

### 步驟 3: 歸檔對話到雙層儲存


In [ ]:
def archive_conversation(self, user_id: str, thread_id: str, messages: list):
    # 生成摘要
    summary = self.generate_summary(messages)
    
    # 提取主題標籤
    topics = self._extract_topics(summary)
    
    # 向量庫只存摘要(用於搜尋)
    self.vectorstore.add_texts(
        texts=[summary],
        metadatas=[{
            "user_id": user_id,
            "thread_id": thread_id,
            "topics": ",".join(topics),
            "timestamp": datetime.now().isoformat()
        }]
    )
    
    # Store 存完整資料(用於詳細檢索)
    self.store.put(("archives", user_id), thread_id, {
        "summary": summary,
        "topics": topics,
        "messages": messages,
        "timestamp": datetime.now().isoformat()
    })

def _extract_topics(self, text: str) -> list[str]:
    """簡單的關鍵字提取(實際應用可使用 NLP 工具)"""
    keywords = ["旅遊", "美食", "工作", "學習", "運動", "電影"]
    return [kw for kw in keywords if kw in text]

**雙層儲存策略**

- Chroma 儲存摘要向量 + metadata ,支援語義搜尋
- InMemoryStore 儲存完整對話,用於獲取詳細資訊
- 兩者透過 thread_id 關聯

### 步驟 4: 記憶搜尋
使用向量相似度搜尋找出相關的歷史對話:

In [ ]:
def search_similar_conversations(self, query: str, user_id: str, k: int = 3):
    """
    使用 Chroma 向量搜尋,召回最相關的長期記憶
    
    參數:
        query: 使用者的當前訊息
        user_id: 使用者 ID(確保只搜尋該使用者的記憶)
        k: 返回最相關的 k 條記憶
    """
    # 向量搜尋
    results = self.vectorstore.similarity_search(
        query, 
        k=k, 
        filter={"user_id": user_id}
    )
    
    memories = []
    for doc in results:
        thread_id = doc.metadata["thread_id"]
        
        # 從 Store 獲取完整資訊
        archived = self.store.get(("archives", user_id), thread_id)
        
        if archived:
            memories.append({
                "thread_id": thread_id,
                "topics": archived.value["topics"],
                "summary": archived.value["summary"]
            })
    
    return memories

**搜尋流程:**

1. 將使用者查詢轉換為向量
2. 在 Chroma 中找出最相似的摘要
3. 使用 thread_id 從 Store 獲取完整資訊
4. 返回結構化的記憶列表

### 步驟 5: 建立 LangGraph 工作流程

定義狀態和節點:

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, Annotated
from operator import add

archive = ConversationArchive()

# 定義狀態結構
class ArchiveState(TypedDict):
    messages: Annotated[list[dict], add]  # 對話訊息(可累加)
    user_id: str                           # 使用者 ID
    thread_id: str                         # 對話執行緒 ID
    similar_conversations: list[dict]      # 召回的相關記憶
    archived: bool                         # 是否已歸檔

# 節點 1: 搜尋歷史記憶
def search_history_node(state: ArchiveState):
    query = state["messages"][-1]["content"]  # 使用最新訊息作為查詢
    user_id = state["user_id"]
    
    similar = archive.search_similar_conversations(query, user_id, k=2)
    
    return {"similar_conversations": similar}

# 節點 2: 基於記憶的對話生成
def chat_with_history_node(state: ArchiveState):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    
    user_msg = state["messages"][-1]["content"]
    memories = state.get("similar_conversations", [])
    
    # 格式化長期記憶
    memory_text = "\n".join(
        f"- {m['summary']}" for m in memories
    ) if memories else "（目前沒有相關的長期記憶）"
    
    # 格式化近期對話
    recent_messages = state["messages"][-6:]  # 只使用最近 6 則訊息
    dialogue_text = "\n".join(
        f"{m['role']}: {m['content']}" for m in recent_messages
    )
    
    # 設計系統提示詞
    system_prompt = f"""
    你是一位有「長期記憶能力」的對話助理。
    
    【長期記憶(來自過往對話摘要)】
    {memory_text}
    
    【近期對話】
    {dialogue_text}
    
    請根據「長期記憶 + 近期對話」,自然地回應使用者。
    - 不要逐字重複摘要
    - 回答要具體、有建議
    - 如果是旅遊問題,可以主動提供選項
    """
    
    response = llm.invoke([
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_msg}
    ])
    
    return {"messages": [{"role": "assistant", "content": response.content}]}

# 節點 3: 歸檔對話
def archive_node(state: ArchiveState):
    # 只有在未歸檔且訊息足夠多時才歸檔
    if not state["archived"] and len(state["messages"]) >= 4:
        archive.archive_conversation(
            state["user_id"], 
            state["thread_id"], 
            state["messages"]
        )
        return {"archived": True}
    return {}

**節點設計說明:**

1. search_history_node: 在每次對話開始時搜尋相關記憶
2. chat_with_history_node: 結合長期記憶和近期對話生成回應
3. archive_node: 在對話結束時歸檔

### 步驟 6: 組裝工作流程

In [ ]:
# 建立工作流程圖
workflow = StateGraph(ArchiveState)

# 添加節點
workflow.add_node("search_history", search_history_node)
workflow.add_node("chat", chat_with_history_node)
workflow.add_node("archive", archive_node)

# 定義節點之間的連接
workflow.set_entry_point("search_history")
workflow.add_edge("search_history", "chat")
workflow.add_edge("chat", "archive")
workflow.add_edge("archive", END)

# 編譯工作流程,加入 checkpointer 以支援狀態持久化
graph = workflow.compile(checkpointer=MemorySaver())

### 步驟 7: 實際使用範例

In [ ]:
# 初始化歸檔系統
archive = ConversationArchive()

user_id = "user_004"

# === 第一次對話 ===
thread_id = "thread_001"
state = {
    "messages": [], 
    "user_id": user_id, 
    "thread_id": thread_id, 
    "similar_conversations": [], 
    "archived": False
}

# 模擬多輪對話
messages = [
    "我想去日本旅遊",
    "我特別想去京都看櫻花"
]

for msg in messages:
    state = graph.invoke(
        {**state, "messages": [{"role": "user", "content": msg}]},
        config={"configurable": {"thread_id": thread_id}}
    )
    print(f"使用者: {msg}")
    print(f"助理: {state['messages'][-1]['content']}\n")

# === 第二次對話(新的 thread,但會召回記憶) ===
thread_id = "thread_002"
state = {
    "messages": [], 
    "user_id": user_id, 
    "thread_id": thread_id, 
    "similar_conversations": [], 
    "archived": False
}

msg = "你覺得春天去哪裡旅遊好?"
result = graph.invoke(
    {**state, "messages": [{"role": "user", "content": msg}]},
    config={"configurable": {"thread_id": thread_id}}
)

print(f"使用者: {msg}")
print(f"助理: {result['messages'][-1]['content']}")

1. 搜尋到之前關於「日本京都櫻花」的對話
2. 在回應中自然地連結到使用者的偏好
3. 可能建議:「根據你之前提到喜歡賞櫻,春天去京都是最佳選擇...」

### 完整程式碼

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore
from typing import TypedDict, Annotated
from operator import add
from datetime import datetime
import os

os.environ["OPENAI_API_KEY"] = "xxx"

# 對話存檔與長期記憶管理
class ConversationArchive:
    def __init__(self):
        # 使用 OpenAI Embeddings
        self.embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
        # 使用 Chroma 建立向量資料庫
        self.vectorstore = Chroma(
            collection_name="conversation_archive",
            embedding_function=self.embeddings
        )
        # InMemoryStore 用來存完整訊息資料
        self.store = InMemoryStore()
        # LLM 用於摘要
        self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    def generate_summary(self, messages: list[dict]) -> str:
        """
        將多則對話濃縮成一句摘要，用於長期記憶。
        """
        conversation = "\n".join(f"{m['role']}: {m['content']}" for m in messages)
        prompt = f"""
        請將以下對話濃縮成一句「可作為長期記憶的摘要」，
        重點放在使用者的意圖、偏好或討論主題：

        {conversation}

        摘要：
        """
        return self.llm.invoke(prompt).content.strip()

    def archive_conversation(self, user_id: str, thread_id: str, messages: list):
        summary = self.generate_summary(messages)
        topics = self._extract_topics(summary)

        # 向量庫只存 summary
        self.vectorstore.add_texts(
            texts=[summary],
            metadatas=[{
                "user_id": user_id,
                "thread_id": thread_id,
                "topics": ",".join(topics),
                "timestamp": datetime.now().isoformat()
            }]
        )

        # Store 存完整資料
        self.store.put(("archives", user_id), thread_id, {
            "summary": summary,
            "topics": topics,
            "messages": messages,
            "timestamp": datetime.now().isoformat()
        })

    def search_similar_conversations(self, query: str, user_id: str, k: int = 3):
        """
        使用 Chroma 向量搜尋，召回最相關的長期記憶摘要
        """
        results = self.vectorstore.similarity_search(query, k=k, filter={"user_id": user_id})
        memories = []
        for doc in results:
            thread_id = doc.metadata["thread_id"]
            archived = self.store.get(("archives", user_id), thread_id)
            if archived:
                memories.append({
                    "thread_id": thread_id,
                    "topics": archived.value["topics"],
                    "summary": archived.value["summary"]
                })
        return memories

    def _extract_topics(self, text: str) -> list[str]:
        keywords = ["旅遊", "美食", "工作", "學習", "運動", "電影"]
        return [kw for kw in keywords if kw in text]

archive = ConversationArchive()

# Workflow State
class ArchiveState(TypedDict):
    messages: Annotated[list[dict], add]
    user_id: str
    thread_id: str
    similar_conversations: list[dict]
    archived: bool  

# Node
def search_history_node(state: ArchiveState):
    query = state["messages"][-1]["content"]
    user_id = state["user_id"]
    similar = archive.search_similar_conversations(query, user_id, k=2)
    return {"similar_conversations": similar}

def chat_with_history_node(state: ArchiveState):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    user_msg = state["messages"][-1]["content"]
    memories = state.get("similar_conversations", [])

    memory_text = "\n".join(f"- {m['summary']}" for m in memories) if memories else "（目前沒有相關的長期記憶）"
    recent_messages = state["messages"][-6:]
    dialogue_text = "\n".join(f"{m['role']}: {m['content']}" for m in recent_messages)

    system_prompt = f"""
    你是一位有「長期記憶能力」的對話助理。

    【長期記憶（來自過往對話摘要）】
    {memory_text}

    【近期對話】
    {dialogue_text}

    請根據「長期記憶 + 近期對話」，自然地回應使用者。
    - 不要逐字重複摘要
    - 回答要具體、有建議
    - 如果是旅遊問題，可以主動提供選項
    """
    response = llm.invoke([
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_msg}
    ])
    return {"messages": [{"role": "assistant", "content": response.content}]}

def archive_node(state: ArchiveState):
    if not state["archived"] and len(state["messages"]) >= 4:
        archive.archive_conversation(state["user_id"], state["thread_id"], state["messages"])
        return {"archived": True}
    return {}

# Workflow 建立
workflow = StateGraph(ArchiveState)
workflow.add_node("search_history", search_history_node)
workflow.add_node("chat", chat_with_history_node)
workflow.add_node("archive", archive_node)
workflow.set_entry_point("search_history")
workflow.add_edge("search_history", "chat")
workflow.add_edge("chat", "archive")
workflow.add_edge("archive", END)
graph = workflow.compile(checkpointer=MemorySaver())

# 範例對話流程
user_id = "user_004"

# 第一次對話
thread_id = "thread_001"
state = {"messages": [], "user_id": user_id, "thread_id": thread_id, "similar_conversations": [], "archived": False}
msgs = ["我想去日本旅遊", "我特別想去京都看櫻花"]
for msg in msgs:
    state = graph.invoke({**state, "messages": [{"role": "user", "content": msg}]},
                         config={"configurable": {"thread_id": thread_id}})

# 第二次對話（召回長期記憶）
thread_id = "thread_002"
state = {"messages": [], "user_id": user_id, "thread_id": thread_id, "similar_conversations": [], "archived": False}
msg = "你覺得春天去哪裡旅遊好？"
result = graph.invoke({**state, "messages": [{"role": "user", "content": msg}]},
                      config={"configurable": {"thread_id": thread_id}})

print(f"\n使用者: {msg}")
print(f"助理: {result['messages'][-1]['content']}")


C:\Users\MyPc\AppData\Local\Temp\ipykernel_3552\148703571.py:29: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)



使用者: 你覺得春天去哪裡旅遊好？
助理: 春天是一個非常適合旅遊的季節，尤其是在日本，特別是京都，賞櫻花的景色絕對讓人難以忘懷。除了京都，你也可以考慮東京的上野公園，或者大阪的櫻花名所如大阪城公園。如果你想要更自然的景色，可以考慮前往富士山周邊的地方，春天的富士山非常迷人。你對哪些地方特別感興趣呢？


### to be continued